In [1]:
# Install necessary packages
%pip install qiskit qiskit-ibm-runtime

# Getting Started with Estimator

### Estimator



In [2]:
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
backend = FakeManilaV2()
print(f"Backend: {backend.name}")

Backend: fake_manila


### Connect to Real Hardware (Optional)

If you have IBM Quantum credentials configured, you can switch to a real backend using `QiskitRuntimeService`.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

try:
    service = QiskitRuntimeService()
    # Select the least busy 127-qubit device
    real_backend = service.least_busy(
        operational=True, simulator=False, min_num_qubits=127
    )
    backend = real_backend # Overwrite the fake backend
    print(f"Connected to real backend: {backend.name}")
except Exception as e:
    print("Qiskit Runtime Service unavailable or no credentials found.")
    print("Using local FakeManilaV2 backend instead.")
    print(f"Error Details: {e}")

In [3]:
from qiskit.circuit.library import qaoa_ansatz
from qiskit.quantum_info import SparsePauliOp
 
entanglement = [tuple(edge) for edge in backend.coupling_map.get_edges() if max(edge) < 4]
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", [i, j], 0.5) for i, j in entanglement],
    num_qubits=4,
)
circuit = qaoa_ansatz(observable, reps=2)
# the circuit is parametrized, so we will define the parameter values for execution
param_values = [0.1, 0.2, 0.3, 0.4]
 
print(f">>> Observable: {observable.paulis}")

>>> Observable: ['IIZZ', 'IIZZ', 'IZZI', 'IZZI', 'ZZII', 'ZZII']


In [4]:
from qiskit.transpiler import generate_preset_pass_manager
 
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

>>> Circuit ops (ISA): OrderedDict({'rz': 44, 'sx': 20, 'cx': 12})


In [5]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator
 
estimator = Estimator(mode=backend)

job = estimator.run([(isa_circuit, isa_observable, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

>>> Job ID: 05e731a4-00b0-45c8-bb84-eb595988a4e0
>>> Job Status: JobStatus.RUNNING


In [6]:
result = job.result()
print(f">>> {result}")
print(f"  > Expectation value: {result[0].data.evs}")
print(f"  > Metadata: {result[0].metadata}")

>>> PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.04356441950417908)), metadata={'target_precision': 0.015625, 'shots': 4096, 'circuit_metadata': {}})], metadata={'version': 2})
  > Expectation value: 1.04443359375
  > Metadata: {'target_precision': 0.015625, 'shots': 4096, 'circuit_metadata': {}}
